# Mathematik für Biologiestudierende

Wintersemester 2025/26

14.01.2026

&copy; 2026 Prof. Dr. Rüdiger W. Braun 

# Wiederholung (interaktiv)

Gehen Sie auf die Website

* https://pingo.coactum.de

und geben Sie folgende Zugangsnummer ein

* **670719**

oder scannen Sie den QR-Code

![QR-Code](bilder/qr02.png)

# Themen heute

* Welch-Test
* Funktionen in Python
* Kruskal-Wallis-Test
* Post-hoc Analyse beim Kruskal-Wallis-Test
* Übersicht Gruppenvergleiche

In [ ]:
import numpy as np
np.set_printoptions(legacy='1.21')
import seaborn as sns
sns.set_theme()
sns.set_context('talk')
import pandas as pd
from scipy import stats

# Posthoc-Analyse beim Alexander-Govern-Test

* ANOVA kann nur im homoskedastischen Fall angewandt werden
* homoskedastisch bedeutet, dass alle Gruppen dieselbe Varianz haben

* im heteroskedastischen Fall wird der Alexander-Govern-Test verwendet

* zur Posthoc-Analyse kann dann nicht der klassische t-Test verwendet werden

# Der Welch-Test

* Der t-Test kann nur gerechnet werden, wenn die Varianzen der zu vergleichenden Datensätze übereinstimmen
* Im heteroskedastischen Fall ist das nicht der Fall
* Man rechnet dann einen Welch-Test
* Der Welch-Test wird auch als "t-Test für ungleiche Varianzen" bezeichnet

* In scipy ist der Welch-Test wie folgt implementiert

* `stats.ttest_ind(x, y, equal_var=False)`

#### Beispiel Barsche

In [ ]:
barsche = pd.read_csv('barsche.csv')
barsche.head()

In [ ]:
ds = barsche[barsche.Art=='gestreift'].Länge
dl = barsche[barsche.Art=='gefleckt'].Länge
db = barsche[barsche.Art=='blau'].Länge
dr = barsche[barsche.Art=='braun'].Länge

In [ ]:
stats.ttest_ind(db, dr, equal_var=False)

* `equal_var=True`: dann wird ein unverbundener t-Test gerechnet
* `equal_var=False`: dann wird ein Welch-Test gerechnet

* Standardeinstellung ist `equal_var=True`

* bei der Posthoc-Analyse wird immer ein zweiseitiger Test gerechnet

* man kann den Welch-Test aber auch wie den t-Test einsetzen, wenn die Gleichheit der Varianzen nicht gesichert ist
* dann sind auch `alternative=greater` und `alternative=less` möglich

### Vergleich von *t*-Test und Welch-Test

* Wenn die Varianzen beider Datensätze gleich sind, dann kann sowohl der $t$-Test als auch der Welch-Test gerechnet werden
* Die Ergebnisse sind dann ähnlich

* Wir vergleichen blaue (`db`) und braune (`dr`) Barsche

In [ ]:
db.var()

In [ ]:
dr.var()

Diese beiden Arten besitzen (annähernd) gleiche Varianz

In [ ]:
stats.ttest_ind(db, dr)    #   t-Test

In [ ]:
stats.ttest_ind(db, dr, equal_var=False)    #  Welch-Test

In [ ]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [ ]:
muc = MultiComparison(barsche.Länge, barsche.Art)

* `muc.allpairtest(stats.ttest_ind)` würde einen t-Test rechnen
* wir können den Schalter `equal_var=False` nicht ohne weiteres übergeben

* wir werden uns ein winziges Programm schreiben

# Python-Funktionen

In [ ]:
def quadrat(x):
    return x*x

In [ ]:
quadrat(5)

* `def`  definiert eine Funktion
* die Rechnungen, aus welchen die Funktion besteht, werden eingerückt
* der auf `return` folgende Wert ist das Funktionsergebnis

# Post-hoc Analyse des Alexander-Govern-Tests

In [ ]:
def welch_test(a, b):
    return stats.ttest_ind(a, b, equal_var=False)

In [ ]:
res = muc.allpairtest(welch_test)[0]
res

# Normalverteilungsannahmen

* Sowohl `f_oneway` als auch `alexandergovern` liefern nur für normalverteilte Daten richtige Ergebnisse
* Normalverteilungsannahmen prüfen wir mit dem Q-Q-Plot

In [ ]:
import statsmodels.api as sm
pp = sm.ProbPlot(db)
pp.qqplot();

das ist fast eine Gerade:  die Daten sind normalverteilt

#### Beispiel Libellen

Flügellängen von Libellen in *mm* (erfundene Daten)

In [ ]:
libellen = pd.read_csv('libellen.csv')
libellen.head()

In [ ]:
sns.boxplot(libellen, x="Art", y="Länge");

In [ ]:
dg = libellen[libellen.Art=='graue'].Länge
du = libellen[libellen.Art=='grüne'].Länge
da = libellen[libellen.Art=='ägyptische'].Länge
dB = libellen[libellen.Art=='Bilker'].Länge

In [ ]:
pp = sm.ProbPlot(dB)
pp.qqplot();

nicht normalverteilt

# Kruskal-Wallis-Test

* im Fall nicht normalverteilter Daten rechnet man den Kruskal-Wallis-Test
* dann ist es auch gleichgültig, ob die Daten heteroskedastisch sind

In [ ]:
stats.kruskal(dg, du, da, dB)

# Post-hoc Analyse

Das nicht-parametrische Analogon zum unverbundenen t-Test ist der Mann-Whitney-Test

In [ ]:
muc = MultiComparison(libellen.Länge, libellen.Art)

In [ ]:
res = muc.allpairtest(stats.mannwhitneyu, method='holm')
res[0]

# Gruppenvergleiche

Übersicht

|Verteilungsannahmen               | Test in `scipy.stats` | Post-hoc-Test        | in `scipy.stats`               |
|:---------------------------------|:----------------------|:---------------------|:-------------------------------|
|normalverteilt, homoskedastisch   | `f_oneway`            | unverbundener t-Test |`ttest_ind`                     |
|normalverteilt, heteroskedastisch | `alexandergovern`     | Welch-Test           |`ttest_ind` mit`equal_var=False`|
|nicht normalverteilt              | `kruskal`             | Mann-Whitney-Test    | `mannwhitneyu`                 |

| Test                 | Art               |
|:---------------------|:------------------|
|t-Test                |parametrisch       |
|Welch-Test            |parametrisch       |
|Mann-Whitney-Test     |nichtparametrisch  |
|ANOVA                 |parametrisch       |
|Alexander-Govern-Test |parametrisch       |
|Kruskal-Wallis-Test   |nichtparametrisch  |

* die parametrischen Tests vergleichen Mittelwerte
* die nichtparametrischen Tests vergleichen Mediane